In [1]:
!nvidia-smi

Sat Nov 30 12:32:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX450         WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   60C    P8             N/A / ERR!  |     182MiB /   2048MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ["CMAKE_ARGS"] = "-DLLAMA_CUBLAS=on"
os.environ["FORCE_CMAKE"] = "1"

In [5]:
!pip install --no-cache-dir llama-cpp-python==0.2.77 \
--force-reinstall --upgrade --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ---------------------------------------- 50.2/50.2 MB 6.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 133.3/133.3 kB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 14.9/14.9 MB 6.6 MB/s eta 0:00:00
     ---------------------------------------- 45.5/45.5 kB ? eta 0:00:00
Failed to build llama-cpp-python


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [50 lines of output]
  *** scikit-build-core 0.10.7 using CMake 3.31.1 (wheel)
  *** Configuring CMake...
  2024-11-30 12:36:17,384 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
  loading initial cache file C:\Users\Abhishek\AppData\Local\Temp\tmpviukx94n\build\CMakeInit.txt
  -- Building for: Visual Studio 17 2022
  -- Selecting Windows SDK version 10.0.22000.0 to target Windows 10.0.22631.
  -- The C compiler identification is MSVC 19.36.32537.0
  -- The CXX compiler identification is MSVC 19.36.32537.0
  -- Detecting C compiler ABI info
  -- Detecting C compiler ABI info - done
  -- Check for working C compiler: C:/Program Files (x86)/Microsoft Visual Studio/2022/BuildTools/VC/Tools/MSVC/14.36.32532/bin/Hostx64/x64/cl.exe - skipped
  -- Detecting C compile features
  -- Detecting C compile features - done
  -- Detecting CXX compiler ABI info
  -- Det

In [4]:
# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the library for data manipulation
import pandas as pd

from tqdm import tqdm # For progress bar related functionalities
tqdm.pandas()

ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
## Model configuration
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=10,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=6000,  # Context window
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

In [ ]:
# function to generate, process, and return the response from the LLM
def generate_llama_response(instruction, user_prompt):

    # System message
    system_message = f"""
    [INST]<<SYS>>
    {instruction}
    <</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{user_prompt}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0.01,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

In [ ]:
instruction_1 = """
    You are an AI analyzing restaurant reviews. Classify the sentiment of the provided review into the following categories:
    - Positive
    - Negative
    - Neutral

    Format the output as a JSON object with a single key-value pair as shown below:
    {"sentiment": "your_sentiment_prediction"}
"""

In [ ]:
review ="""
This film is one of the all-time classics -- it won Academy Awards for Best Screenplay, Best Cinematography, Best Musical Score, and Best Original Song; it was nominated for Best Sound, Best Director, and Best Picture.  This movie is about as well put together as a movie can get. Legendary screenwriter, William Goldman, gifts the characters with sharp, witty, quotable dialogue which Paul Newman and Robert Redford deliver with deadpan perfection.  What little time the supporting cast gets, they use well (with a particular emphasis on Strother Martin's unforgettably "colorful" performance as Percy Garris.  Burt Bacharach's 60s-vibe score sounds a little odd for a western, but it offers a tone that period music just wouldn't deliver. The real gem of this film, though, is Conrad Hall's photography.  Intricately and beautifully crafted, the images are the main storytelling vocabulary.  There is heart, metaphor, character, and action in every frame of this masterpiece.  Film buffs and other careful viewers are treated to subtle nuances in lighting, angle, framing, and composition that enhance each line of dialogue, each action sequence, and the theme as a whole.  Stunning.
"""

In [ ]:
generate_llama_response(instruction_1, review)


llama_print_timings:        load time =   35964.77 ms
llama_print_timings:      sample time =     239.66 ms /   357 runs   (    0.67 ms per token,  1489.59 tokens per second)
llama_print_timings: prompt eval time =   35963.72 ms /   404 tokens (   89.02 ms per token,    11.23 tokens per second)
llama_print_timings:        eval time =  367376.21 ms /   356 runs   ( 1031.96 ms per token,     0.97 tokens per second)
llama_print_timings:       total time =  404205.19 ms /   760 tokens


' Sure, I\'d be happy to help! Here is the review you provided:\n\n"This film is one of the all-time classics -- it won Academy Awards for Best Screenplay, Best Cinematography, Best Musical Score, and Best Original Song; it was nominated for Best Sound, Best Director, and Best Picture.  This movie is about as well put together as a movie can get. Legendary screenwriter, William Goldman, gifts the characters with sharp, witty, quotable dialogue which Paul Newman and Robert Redford deliver with deadpan perfection.  What little time the supporting cast gets, they use well (with a particular emphasis on Strother Martin\'s unforgettably "colorful" performance as Percy Garris.  Burt Bacharach\'s 60s-vibe score sounds a little odd for a western, but it offers a tone that period music just wouldn\'t deliver. The real gem of this film, though, is Conrad Hall\'s photography.  Intricately and beautifully crafted, the images are the main storytelling vocabulary.  There is heart, metaphor, characte

In [ ]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    import json
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_category = json_str[json_start:json_end + 1]  # Extract the JSON object
            data_dict = json.loads(extracted_category)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

In [ ]:
data = pd.read_csv('stock_news.csv')

In [ ]:
data["Date"] = pd.to_datetime(data['Date'])  # Convert the 'Date' column to datetime format.

In [ ]:
# Group the data by week using the 'Date' column.
weekly_grouped = data.groupby(pd.Grouper(key='Date', freq='W'))

In [ ]:
weekly_grouped = weekly_grouped.agg(
    {
        'News': lambda x: ' || '.join(x)  # Join the news values with ' || ' separator.
    }
).reset_index()

print(weekly_grouped.shape)

(18, 2)


In [ ]:
# creating a copy of the data
data_1 = weekly_grouped.copy()

In [ ]:
prompt = """You are an expert data analyst specializing in news content analysis.
Analyse the provided news headlines carefully and the return the top-3 positive events and top-3 negative events that impact the stock prices.

Format the output as a JSON object with two key-value pairs as shown below:
{"Positive Events": "[your top-3 positive events prediction]", "Negative Events": "[your top-3 positive events prediction]"}
"""

news_text = data_1['News'][0]

In [ ]:
news_text1 = " || ".join(news_text.split(" || ")[:2])

In [ ]:
llm_response = generate_llama_response(prompt, news_text)

Llama.generate: prefix-match hit


In [ ]:
extract_json_data(llm_response)